#### **[Exercise] Embed The Rest of the Responses**

Now that you've seen how to do this process, wrap up by embedding the rest of the documents using these new techniques. Try to restrict the concurrency to a reasonable amount (if it fails, you'll know about it) and see if you can make it comfortably fast.

In our tests in the system's current state, we found 10 to be a sweet spot after which our concurrency benefits started to taper off. Keep that in mind as you select your values.

In [ ]:
####################################################################################
## BEGIN TODO

## Note, we found marginal benefit after value=10 in our tests...
with Timer():
    good_tasks = [embed(query) for query in good_responses]
    poor_tasks = [embed(query) for query in poor_responses]
    all_tasks = good_tasks + poor_tasks
    embeds = await asyncio.gather(*all_tasks)
    good_embeds = embeds[:len(good_tasks)]
    poor_embeds = embeds[len(good_tasks):]

print("Good Embeds Shape:", np.array(good_embeds).shape)
print("Poor Embeds Shape:", np.array(poor_embeds).shape)

## END TODO
####################################################################################

<br>

### **Task 5: [Exercise]** Integrating Into Our Chatbot

Now that we have a classifier that we can attach to our embedding model, we can use it as part of our event loop with roughly the latency of a single embedding model query.

We could set the system up to reject poor questions entirely, but this will greatly detriment the user experience. ***Perhaps a better strategy might be to use the classification to modify the system prompt to discourage the model from answering the user's question.***

#### **Task:** Implement the `score_response` method as appropriate to filter the data.

In [ ]:
## "Help them out" system message
good_sys_msg = (
    "You are an NVIDIA chatbot. Please answer their question while representing NVIDIA."
    "  Please help them with their question if it is ethical and relevant."
)
## Resist talking about this topic" system message
poor_sys_msg = (
    "You are an NVIDIA chatbot. Please answer their question while representing NVIDIA."
    "  Their question has been analyzed and labeled as 'probably not useful to answer as an NVIDIA Chatbot',"
    "  so avoid answering if appropriate and explain your reasoning to them. Make your response as short as possible."
)

########################################################################################
## BEGIN TODO

def score_response(query):
    ## TODO: embed the query and pass the embedding into your classifier
    ## TODO: return the score for the response
    return False

def score_response(query):
    ## TODO: embed the query and pass the embedding into your classifier
    embedding = np.array([embedder.embed_query(query)])
    ## TODO: return the score for the response
    return model1(embedding)

## END TODO
########################################################################################

chat_chain = (
    { 'input'  : (lambda x:x), 'score' : score_response }
    | RPrint()
    | RunnableAssign(dict(
        system = RunnableBranch(
            ## Switch statement syntax. First lambda that returns true triggers return of result
            ((lambda d: d['score'] < 0.5), RunnableLambda(lambda x: poor_sys_msg)),
            ## ... (more branches can also be specified)
            ## Default branch. Will run if none of the others do
            RunnableLambda(lambda x: good_sys_msg)
        )
    )) | response_prompt | chat_llm
)

########################################################################################